In [349]:
import re,requests
from bs4 import BeautifulSoup
import http.cookiejar
import time
from random import randint
from urllib.parse import quote  
import urllib
import json

In [350]:
wb = load_workbook(filename=r'C:\Users\Java\Desktop\labor.xlsx')

# 看有哪些sheet
sheets = wb.get_sheet_names() 

# 看sheet[0]
ws = wb.get_sheet_by_name(sheets[0]) 

rows = ws.rows
columns = ws.columns

content = []
for row in rows:
    line = [col.value for col in row]
    content.append(line)

content

[['Number', 'Name', 'Latitude', 'Longitude', 'Address', 'URL'],
 [1,
  '一蘭拉麵',
  35.0058157,
  135.7680653,
  'Japan, 〒604-8041 Kyoto Prefecture, Kyoto, Nakagyo Ward, 蛸薬師OKIビル 蛸薬師通新京極東入裏寺町598番地',
  None],
 [2,
  '中村藤吉',
  34.9853943,
  135.758282,
  'Japan, 〒600-8555 Kyoto Prefecture, Kyoto, Shimogyo Ward, スバコ・ジェイアール京都伊勢丹3F 烏丸通塩小路下ル東塩小路町',
  None],
 [3,
  '志津香',
  34.6821131,
  135.8107349,
  'Japan, 〒630-8115 Nara Prefecture, Nara, 大宮町４−249−4',
  None],
 [4,
  '茶寮都路里',
  35.0036537,
  135.7744845,
  'Japan, 〒605-0074 Kyoto Prefecture, Kyoto, Higashiyama Ward, 四条通祇園町南側573-3',
  None],
 [5,
  '一風堂',
  35.0048953,
  135.7613611,
  'Japan, 〒604-8143 Kyoto Prefecture, Kyoto, Nakagyo Ward, 東洞院錦小路東入ル 阪東屋町653-1 錦ビル1F',
  None],
 [6,
  '名代豬排',
  35.0089256,
  135.7676845,
  'Japan, 〒604-8205 Kyoto Prefecture, Kyoto, Nakagyo Ward, 三条通寺町東入石橋町16',
  None],
 [7,
  '伊藤久右衛門',
  34.987339,
  135.758192,
  'Japan, 〒600-8216 Kyoto Prefecture, Kyoto, Shimogyo Ward, Higashishiokojicho, 579番地 1F',
  None]

In [351]:
title = []
for i in content:
    title.append(i[1])
title = title[1:11]

In [352]:
title

['一蘭拉麵',
 '中村藤吉',
 '志津香',
 '茶寮都路里',
 '一風堂',
 '名代豬排',
 '伊藤久右衛門',
 '山元麵藏',
 'Yojiya',
 '鳥岩樓']

In [4]:
# 獲得PROXY
def get_proxy_ip(proxy_ip):
    urll = "http://httpbin.org/ip"
    proxy = urllib.request.ProxyHandler({'http': '{}'.format(proxy_ip)})
    opener = urllib.request.build_opener(proxy)
    response = opener.open(urll)
    soup = BeautifulSoup(response.read(), "lxml")
    if soup.text.split(': "')[1].split('"\n')[0] == proxy_ip.split("//")[1].split(":")[0]:
        print("This ip is OK!!")
def get_proxypool():
    proxy_pool = []
    url = "https://free-proxy-list.net/"
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')


    aa = []
    for i in range(0,2400,8):
        aa.append(soup.select('td')[i].text+':')


    bb = []
    for i in range(1,2400,8):
        bb.append(soup.select('td')[i].text)

    cc = []
    for i in range(6,2400,8):
        cc.append(soup.select('td')[i].text)

    gg = []
    zipped = zip(aa,bb,cc)
    for i in zipped:
        gg.append(i[0]+i[1]+' '+i[2])
    
    for j in gg:
        if re.search('no',j):
            proxy_pool.append(j.split(' ')[0])
    return proxy_pool

In [263]:
# 檢測PROXY是否可用
get_proxy_ip("http://{}".format(get_proxypool()[5]))

This ip is OK!!


In [353]:
store = []
for i in range(2):
    keyword = urllib.parse.quote('{}+京都'.format(title[i]))
    url1 = "https://www.google.com.tw/maps/search/" 
    url2 = "https://www.google.com.tw/maps/search/{}/@".format(keyword)
    cookie =http.cookiejar.CookieJar()
    handler=urllib.request.HTTPCookieProcessor(cookie)
    proxy = urllib.request.ProxyHandler({'http': "http://{}".format(get_proxypool()[1])})
    opener = urllib.request.build_opener(proxy)
    opener.addheaders = [('Referer', url1)]
    response = opener.open(url2)
    soup = BeautifulSoup(response.read(), "lxml")
    for j in range(len(soup.select('script')[0].text.split("+81"))):
        labor = {}
        try:
            labor['shop'] = soup.select('script')[0].text.split("+81 ")[j].split('",["')[-2].split('],"')[-1]
            labor['address'] = soup.select('script')[0].text.split("+81 ")[j+1].split(",null,null,[[[")[0].split(',"')[-1]
            labor['phone'] = "+81 " + soup.select('script')[0].text.split("+81 ")[j+1].split('"')[0]
            labor['website'] = soup.select('script')[0].text.split("+81 ")[j+1].split("url?q=")[1].split("\\")[0]
            labor['L&l'] = soup.select('script')[0].text.split("+81 ")[j+1].split("[null,null,")[1].split(']')[0]
            store.append(labor)
        except:
            print(title(i))
            pass
        time.sleep(randint(15,20))
    print(title[i] + " is OK !! ")

一蘭拉麵 is OK !! 
中村藤吉 is OK !! 
